In [7]:
from datasets import Dataset
dataset = Dataset.load_from_disk("/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final")
answers = dataset['cop']

cnt = 0
for ans in answers:
    if ans not in [0,1,2,3]:
        cnt+=1
print(cnt)

243


## run baseline

In [3]:
query_file = "/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final/queries.txt"
# Read queries from the file
with open(query_file, "r") as file:
    queries = file.readlines()

print(queries[900])

Absorption of Glucose in the intestine is by:: A. Primary Active Transpo, B. Secondary Active Transpo, C. Facilitated Diffusion, D. Simple Diffusion. Please select the correct answer from A, B, C, D. Put your answer in \boxed{}.



In [4]:
import os

# File paths
query_file = "/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final/queries.txt"
response_file = "/scratch/gpfs/jx0800/data/graphrag/results/baseline_32k_8B.log"

system_prompt = r"""---Role---

You are a helpful assistant responding to questions about data in the tables provided.

---Target response format---

Answer in the following format:
Analysis of the question: 
- Understand the question and find out what is needed to answer it
- Analyze and answer the question

Answer:
- Provide a concise answer using `\boxed{}`, select only the correct letter (e.g., \boxed{C}) 
"""

from openai import OpenAI

client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY")
)


# Function to call OpenAI API
def call_api(query):
    response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct", #meta-llama/Llama-3.3-70B-Instruct
    max_tokens=1024,
    temperature=0.0,
    top_p=0.9,
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ]
    )
    return response.choices[0].message.content

# Read queries from the file
with open(query_file, "r") as file:
    queries = file.readlines()

# Process each query and save responses
with open(response_file, "w") as output_file:
    for i, query in enumerate(queries):
        # if i <= 99: 
        #     continue
        query = query.strip()  # Remove leading/trailing whitespace
        if query:  # Skip empty lines
            print(f"Processing query {i}:")
            response = call_api(query)
            output_file.write(f"Query {i}: {query}\n")
            output_file.write(f"Response: {response}\n")
            output_file.write("-" * 50 + "\n")  # Separator for readability
        if i == 99: 
            break

print("All queries processed and responses saved to", response_file)

Processing query 0:
Processing query 1:
Processing query 2:
Processing query 3:
Processing query 4:
Processing query 5:
Processing query 6:
Processing query 7:
Processing query 8:
Processing query 9:
Processing query 10:
Processing query 11:
Processing query 12:
Processing query 13:
Processing query 14:
Processing query 15:
Processing query 16:
Processing query 17:
Processing query 18:
Processing query 19:
Processing query 20:
Processing query 21:
Processing query 22:
Processing query 23:
Processing query 24:
Processing query 25:
Processing query 26:
Processing query 27:
Processing query 28:
Processing query 29:
Processing query 30:
Processing query 31:
Processing query 32:
Processing query 33:
Processing query 34:
Processing query 35:
Processing query 36:
Processing query 37:
Processing query 38:
Processing query 39:
Processing query 40:
Processing query 41:
Processing query 42:
Processing query 43:
Processing query 44:
Processing query 45:
Processing query 46:
Processing query 47:
Pr

## calculate accuracy

### define utils functions

In [1]:
def get_answers(log_file, split_on='-'*50, answer_note = 'SUCCESS: Local Search Response:'):
    
    with open(log_file, 'r') as file:
        output_content = file.read()

    # Split the responses based on the separator
    responses = output_content.strip().split(split_on)

    # Regular expression to find the part after 'SUCCESS: Local Search Response:'
    pattern = fr'{re.escape(answer_note)}(.*)'
    success_pattern = re.compile(pattern, re.DOTALL)

    # Regular expression to find the content inside \boxed{}
    boxed_pattern = re.compile(r'\\boxed\{([^}]+)\}')
    box_pattern = re.compile(r'\\box\{([^}]+)\}')

    answers = []
    # Process each response
    for i, response in enumerate(responses):
        
        # Find the part after 'SUCCESS: Local Search Response:'
        success_match = success_pattern.search(response)
        if success_match:
            success_text = success_match.group(1).strip()
            #print(f"SUCCESS Text: {success_text}")
            
            # Check if there's an answer in \boxed{}
            boxed_match = boxed_pattern.search(success_text)
            if boxed_match:
                answer = boxed_match.group(1)
                answers.append(answer)
                #print(f"Answer found in \\boxed{{}}: {answer}")
            else:
                box_match = box_pattern.search(success_text)
                if box_match:
                    answer = box_match.group(1)
                    answers.append(answer)
                else:
                    answers.append('E')
                    print(f"Processing Response {i}:")
                    print("No answer found in \\boxed{}.")
                    print(success_text)
        else:
            answers.append(None)
            print(f"Processing Response {i}:")
            print("No 'SUCCESS: Local Search Response:' found in this response.")
    
    return answers
    

# Function to process each response
def process_answer(answer):
    # Mapping of valid responses
    response_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    # Check if the response starts with A, B, C, or D
    if answer[0] in response_mapping:
        # Extract the first character (A, B, C, or D)
        key = answer[0]
        return response_mapping[key]
    else:
        # Handle invalid responses
        return 4

def accuracy_score(y_true, y_pred):
    correct = 0
    wrong_indices = []
    
    for idx, (true, pred) in enumerate(zip(y_true, y_pred)):
        if true == pred:
            correct += 1
        else:
            wrong_indices.append(idx)
    
    accuracy = correct / len(y_true) if y_true else 0
    return accuracy, wrong_indices

## 8B model injected KG

In [2]:
import re

log_file_seed = "/scratch/gpfs/jx0800/data/graphrag/results/injected_32k_8B.log"

answers = get_answers(log_file_seed, split_on='-'*40, answer_note = 'SUCCESS: Local Search Response:')
print(answers)
print()

answers = answers[:100]
processed_answers = [process_answer(answer) for answer in answers]

print(f"processed_answers: {processed_answers}")
print()

import json
answer_file = "/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final/answers.json"
with open(answer_file, "r") as file:
    true_answers = json.load(file)

true_answers = true_answers[0:len(processed_answers)]

accuracy, wrong_indices_seed = accuracy_score(true_answers, processed_answers)

# Print the result
print(f"Accuracy: {accuracy:.4f}")
print("wrong_indices:", wrong_indices_seed)
response_mapping = {0:'A', 1:'B', 2:'C', 3:'D', 4:'None'}
print("wrong answer for these indices:", [response_mapping[processed_answers[idx]] for idx in wrong_indices_seed])
print("true answer for these indices:", [true_answers[idx] for idx in wrong_indices_seed])

Processing Response 7:
No answer found in \boxed{}.
**Summary of the Input Data Table**:  
The input data table contains information about various transplantation procedures, including pancreas and islet cell transplants. However, there is no specific information about the site of transplantation for islet cell transplants for diabetes mellitus. 

**Analysis of the question with references**: 
To answer this question, we need to find information about islet cell transplants in the input data table. Unfortunately, there is no direct reference to islet cell transplants in the Entities table. However, we can look for related information in the Relationships table.

Upon reviewing the Relationships table, we find that there is a mention of "posttransplant diabetes mellitus" in the Entities table (record id: 1242) and a relationship between "transplantation" and "posttransplant diabetes mellitus" in the Relationships table (record id: 9331). However, this information does not provide specif

## 8B model injected KG, 10 entities

In [14]:
import re

log_file_seed = "/scratch/gpfs/jx0800/data/graphrag/results/injected_32k_query_output_8B_10_entities.log"

answers = get_answers(log_file_seed, split_on='-'*40, answer_note = 'SUCCESS: Local Search Response:')
print(answers)
print()

answers = answers[:100]
processed_answers = [process_answer(answer) for answer in answers]

print(f"processed_answers: {processed_answers}")
print()

import json
answer_file = "/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final/answers.json"
with open(answer_file, "r") as file:
    true_answers = json.load(file)

true_answers = true_answers[0:len(processed_answers)]

accuracy, wrong_indices_seed = accuracy_score(true_answers, processed_answers)

# Print the result
print(f"Accuracy: {accuracy:.4f}")
print("wrong_indices:", wrong_indices_seed)
response_mapping = {0:'A', 1:'B', 2:'C', 3:'D', 4:'None'}
print("wrong answer for these indices:", [response_mapping[processed_answers[idx]] for idx in wrong_indices_seed])

Processing Response 19:
No answer found in \boxed{}.
I don't have information about the primary etiologic factor in the development of furcation defect. The provided data tables do not contain any information related to furcation defect or its etiology.
Processing Response 25:
No answer found in \boxed{}.
**Summary of the Input Data Table**:  
The input data table contains information about various eye disorders and infections associated with diabetes. The "Entities" table lists different types of eye disorders, including fungal orbital cellulitis, while the "Relationships" table describes the relationships between these disorders and other conditions. 

**Analysis of the question with references**: 
The question asks about the most common cause of fungal orbital cellulitis in diabetes. To answer this, we need to look for relationships between fungal orbital cellulitis and other conditions in the "Relationships" table.

Upon analyzing the "Relationships" table, we find that there is no

## 8B model injected KG, 30 entities

In [2]:
import re

log_file_seed = "/scratch/gpfs/jx0800/data/graphrag/results/injected_32k_8B_30_entities.log"

answers = get_answers(log_file_seed, split_on='-'*40, answer_note = 'SUCCESS: Local Search Response:')
print(answers)
print()

answers = answers[:100]
processed_answers = [process_answer(answer) for answer in answers]

print(f"processed_answers: {processed_answers}")
print()

import json
answer_file = "/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final/answers.json"
with open(answer_file, "r") as file:
    true_answers = json.load(file)

true_answers = true_answers[0:len(processed_answers)]

accuracy, wrong_indices_seed = accuracy_score(true_answers, processed_answers)

# Print the result
print(f"Accuracy: {accuracy:.4f}")
print("wrong_indices:", wrong_indices_seed)
response_mapping = {0:'A', 1:'B', 2:'C', 3:'D', 4:'None'}
print("wrong answer for these indices:", [response_mapping[processed_answers[idx]] for idx in wrong_indices_seed])

Processing Response 15:
No answer found in \boxed{}.

Processing Response 78:
No answer found in \boxed{}.
Unfortunately, I don't have information about Peptide YY being secreted by any of the options provided. However, I can tell you that Peptide YY is a peptide hormone that is secreted by the L cells in the ileum of the small intestine. 

Since this information is not present in the provided data tables, I will not be able to provide a correct answer in the format requested.
Processing Response 100:
No 'SUCCESS: Local Search Response:' found in this response.
['A', 'C', 'D', 'C', 'A', 'A', 'B', 'None of the above', 'D', 'A', 'A', 'A', 'A', 'C', 'B', 'E', 'C', 'D', 'D', 'B', 'D', 'D', 'D', 'D', 'D', 'B. bc', 'D', 'A', 'A', 'D', 'C', 'C', 'B', 'A', 'C', 'B', 'D', 'C', 'C', 'C', 'B', 'B', 'D', 'B', 'A', 'C', 'B', 'B', 'C', 'C', 'C', 'B', 'C', 'D', 'B', 'A', 'B', 'B', 'D', 'B', 'A', 'D', 'D', 'B', 'C', 'B', 'B', 'A', 'C', 'C', 'D', 'D', 'A', 'B', 'C', 'D', 'B', 'B', 'E', 'B', 'B', 'A', '

## 8B injected expanded KG

In [3]:
import re

log_file_seed = "/scratch/gpfs/jx0800/data/graphrag/results/expanded_32k_8B.log"

answers = get_answers(log_file_seed, split_on='-'*40, answer_note = 'SUCCESS: Local Search Response:')
print(answers)
print()

answers = answers[:100]
processed_answers = [process_answer(answer) for answer in answers]

print(f"processed_answers: {processed_answers}")
print()

import json
answer_file = "/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final/answers.json"
with open(answer_file, "r") as file:
    true_answers = json.load(file)

true_answers = true_answers[0:len(processed_answers)]

accuracy, wrong_indices = accuracy_score(true_answers, processed_answers)

# Print the result
print(f"Accuracy: {accuracy:.4f}")
print("wrong_indices:", wrong_indices)
response_mapping = {0:'A', 1:'B', 2:'C', 3:'D', 4:'None'}
print("wrong answer for these indices:", [response_mapping[processed_answers[idx]] for idx in wrong_indices])

Processing Response 15:
No answer found in \boxed{}.

Processing Response 32:
No answer found in \boxed{}.
I don't have enough information to answer this question. The input data tables do not contain any information about the medications listed (Zidovudine, Lamivudine, Stavudine, Didanosine) or their association with peripheral neuropathy.

However, based on general knowledge, Stavudine (D) is known to be associated with peripheral neuropathy as a side effect.
Processing Response 51:
No answer found in \boxed{}.
**Summary of the Input Data Table**:  
The input data table contains information about various entities related to pregnancy and diabetes, including fasting glucose levels. However, there is no specific information provided about the target fasting blood sugar range for pregnant diabetic females.

**Analysis of the question with references**: 
To answer this question, we need to look for information related to fasting glucose levels in pregnant diabetic females. Unfortunately,

In [5]:
inj = [2, 7, 8, 10, 13, 15, 17, 18, 22, 25, 28, 30, 31, 32, 35, 39, 43, 51, 55, 56, 61, 63, 68, 69, 80, 82, 83, 85, 90, 94, 97, 98] #[1, 2, 4, 8, 10, 12, 13, 15, 16, 17, 19, 21, 24, 26, 27, 28, 32, 34, 36, 41, 42, 45, 47, 49, 52, 53, 56, 58, 59, 64, 65, 67, 70, 72, 74, 76, 79, 86, 87, 96, 99]
exp = [2, 6, 8, 15, 17, 18, 22, 25, 28, 30, 31, 32, 34, 39, 41, 50, 51, 53, 55, 56, 61, 68, 74, 77, 78, 82, 83, 90, 94, 96, 97, 98] #[0, 1, 4, 7, 8, 13, 15, 16, 17, 21, 23, 24, 25, 26, 27, 32, 36, 39, 40, 47, 48, 52, 53, 56, 59, 61, 62, 65, 70, 71, 72, 74, 76, 81, 83, 85, 86, 87, 88, 96, 99]
base = [6, 8, 17, 25, 31, 43, 51, 61, 68, 82, 83, 85, 87, 90, 94, 97, 98] #[0, 1, 2, 8, 10, 12, 13, 17, 22, 24, 26, 28, 36, 41, 47, 65, 72, 74, 76, 83, 86, 99]

# calculate overlap percentage
# Compute intersection
intersection1 = set(inj) & set(exp)
intersection2 = set(inj) & set(base)
intersection3 = set(exp) & set(base)
# Compute overlap percentage
overlap_percentage = (len(intersection1) / min(len(inj), len(exp))) * 100
print(overlap_percentage)
overlap_percentage = (len(intersection2) / min(len(inj), len(base))) * 100
print(overlap_percentage)
overlap_percentage = (len(intersection3) / min(len(exp), len(base))) * 100
print(overlap_percentage)

71.875
88.23529411764706
82.35294117647058


refuse to answer: 2   (38, 60)
Don't have data in the table: 5

## calculate accuracy of baseline 8B model

In [5]:
import re

log_file_baseline = "/scratch/gpfs/jx0800/data/graphrag/results/baseline_32k_8B.log"

answers = get_answers(log_file_baseline, split_on='-'*50, answer_note = 'Response: ')
print(f"Answers: {answers}")
print(f"Number of Answers: {len(answers)}")

answers = answers[:100] # calculate first 100 acc
processed_answers = [process_answer(answer) for answer in answers]

print(f"processed_answers: {processed_answers}")

import json
answer_file = "/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final/answers.json"
with open(answer_file, "r") as file:
    true_answers = json.load(file)

true_answers = true_answers[0:len(processed_answers)]

accuracy, wrong_indices_baseline = accuracy_score(true_answers, processed_answers)

# Print the result
print(f"Accuracy: {accuracy:.4f}")
print("wrong_indices:", wrong_indices_baseline)
response_mapping = {0:'A', 1:'B', 2:'C', 3:'D', 4:'None'}
print("wrong answer for these indices:", [response_mapping[processed_answers[idx]] for idx in wrong_indices_baseline])

Processing Response 100:
No 'SUCCESS: Local Search Response:' found in this response.
Answers: ['A', 'C', 'A', 'C', 'A', 'A', 'D', 'D', 'D', 'A', 'A', 'D', 'A', 'B', 'B', 'B', 'C', 'D', 'D', 'B', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'A', 'B', 'D', 'C', 'B', 'C', 'A', 'C', 'B', 'D', 'C', 'C', 'D', 'B', 'B', 'D', 'B', 'B', 'C', 'B', 'B', 'C', 'C', 'C', 'B', 'C', 'D', 'B', 'A', 'D', 'B', 'D', 'B', 'A', 'C', 'D', 'B', 'C', 'B', 'B', 'A', 'C', 'A', 'D', 'D', 'A', 'B', 'C', 'D', 'B', 'B', 'C', 'B', 'D', 'A', 'C', 'C', 'C', 'A', 'D', 'B', 'A', 'A', 'D', 'B', 'B', 'D', 'B', 'D', 'D', 'D', 'A', 'B', None]
Number of Answers: 101
processed_answers: [0, 2, 0, 2, 0, 0, 3, 3, 3, 0, 0, 3, 0, 1, 1, 1, 2, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 0, 1, 3, 2, 1, 2, 0, 2, 1, 3, 2, 2, 3, 1, 1, 3, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 3, 1, 0, 3, 1, 3, 1, 0, 2, 3, 1, 2, 1, 1, 0, 2, 0, 3, 3, 0, 1, 2, 3, 1, 1, 2, 1, 3, 0, 2, 2, 2, 0, 3, 1, 0, 0, 3, 1, 1, 3, 1, 3, 3, 3, 0, 1]
Accuracy: 0.8300
wrong_indices: [6, 8, 17, 25, 31, 43, 

## calculate accuracy of baseline 8B model on 1000 questions

In [6]:
import re

log_file_baseline = "/scratch/gpfs/jx0800/data/graphrag/results/baseline_responses_llama8B.log"

answers = get_answers(log_file_baseline, split_on='-'*50, answer_note = 'Response: ')
print(f"Answers: {answers}")
print(f"Number of Answers: {len(answers)}")

answers = answers[:1000] # calculate first 100 acc
processed_answers = [process_answer(answer) for answer in answers]

print(f"processed_answers: {processed_answers}")

import json
answer_file = "/projects/JHA/shared/dataset/mcq_filtered_by_match/answers.json"
with open(answer_file, "r") as file:
    true_answers = json.load(file)

true_answers = true_answers[0:len(processed_answers)]

accuracy, wrong_indices_baseline = accuracy_score(true_answers, processed_answers)

# Print the result
print(f"Accuracy: {accuracy:.4f}")
print("wrong_indices:", wrong_indices_baseline)
response_mapping = {0:'A', 1:'B', 2:'C', 3:'D', 4:'None'}
print("wrong answer for these indices:", [response_mapping[processed_answers[idx]] for idx in wrong_indices_baseline])

Processing Response 553:
No answer found in \boxed{}.
Analysis of the question:
- The question asks for the exception to the diagnosis of all conditions listed by fluorescein angiography.

Analysis of options:
- Fluorescein angiography is used to diagnose various retinal conditions by injecting a fluorescent dye into the bloodstream and taking photographs as it circulates through the retina.
- Diabetes retinopathy, hypertensive retinopathy, and choroidal neovascularization are all conditions that can be diagnosed with fluorescein angiography.
- Central serous retinopathy is typically diagnosed with fluorescein angiography, as the condition involves the accumulation of fluid under the retina, which can be visualized with this technique.

However, the correct answer is B. Hypeensive retinopathy is not typically diagnosed with fluorescein angiography. However, hypertensive retinopathy is diagnosed with fundoscopy and OCT imaging.
Processing Response 1000:
No 'SUCCESS: Local Search Respons

### check wrong responses

In [10]:
def display_wrong_answers(log_file, id_list, split_on='-'*50, answer_note = 'SUCCESS: Local Search Response:'):
    with open(log_file, 'r') as file:
        output_content = file.read()

    # Split the responses based on the separator
    responses = output_content.strip().split(split_on)

    # Regular expression to find the part after 'SUCCESS: Local Search Response:'
    pattern = fr'{re.escape(answer_note)}(.*)'
    success_pattern = re.compile(pattern, re.DOTALL)
    
    answers = []

    for idx in id_list:
        success_match = success_pattern.search(responses[idx])
        if success_match:
            success_text = success_match.group(1).strip()
            answers.append(success_text)
            print(f"{idx}: {success_text}\n")
        else:
            answers.append(None)
            print(f"{idx}: None\n")

    return #answers
    

In [11]:
display_wrong_answers(log_file_seed, wrong_indices_seed, split_on='-'*40)#wrong_indices_baseline

2: **Summary of the Input Data Table**:  
- The input data table contains information about various disorders and procedures related to the abdomen and pancreas.
- There are several entities related to pancreatitis and pancreatic disorders, such as pancreatitis, pseudocyst of pancreas, and pancreaticoduodenostomy.
- The relationships between these entities are also provided, including the location of procedures and the effects of certain substances on pancreatitis.

**Analysis of the question with references**: 
- The question describes a patient with chronic pancreatitis, as evidenced by abdominal pain radiating to the back, dilated pancreatic duct, and stones in the tail of the pancreas.
- The patient's symptoms and findings suggest a need for intervention to address the pancreatic disease.

**Answer**:
- Based on the input data table, the most appropriate management for this patient would be surgical intervention to address the pancreatic disease.
- Option B, Pancreatico Jejunostomy

In [13]:
display_wrong_answers(log_file_baseline, wrong_indices_seed, answer_note = 'Response: ')

2: **Analysis of the question**:
The question is asking for the most appropriate management for a patient with chronic alcoholism, abdominal pain radiating to the back, dilated pancreatic duct, and stones in the tail of the pancreas.

**Answer**:
\boxed{B}

4: **Analysis of the question**:
- The patient presents with symptoms of nausea, vomiting, and abdominal pain, and has X-ray findings of centrally located lytic lesions in the metacarpals.
- The question is asking for the next line of management for this patient.

**Answer**:
\boxed{D}

5: **Analysis of the question**:
- The question is asking about the characteristics of post spinal headache, specifically which of the given statements is not true.
- The options provided are related to the timing, position, cause, and associated symptoms of post spinal headache.

**Answer**:
\boxed{A}

7: **Analysis of the question**:
The question requires identifying the most likely diagnosis for a 1-month-old baby presenting with frequent vomiting

In [12]:
display_wrong_answers(log_file_baseline, wrong_indices_baseline, answer_note = 'Response: ')

2: **Analysis of the question**:
The question is asking for the most appropriate management for a patient with chronic alcoholism, abdominal pain radiating to the back, dilated pancreatic duct, and stones in the tail of the pancreas.

**Answer**:
\boxed{B}

5: **Analysis of the question**:
- The question is asking about the characteristics of post spinal headache, specifically which of the given statements is not true.
- The options provided are related to the timing, position, cause, and associated symptoms of post spinal headache.

**Answer**:
\boxed{A}

10: **Analysis of the question**:
- The patient's symptoms suggest a possible issue with the lacrimal gland, which produces tears.
- The history of recurrent bacterial infections might indicate a problem with the immune system.
- Retinol binding protein (RBP) is a protein that transports retinol (vitamin A) in the plasma.

**Answer**:
\boxed{B}

11: **Analysis of the question**:
The question is asking about the effect of aspirin in v

In [61]:
with open(query_file, "r") as file:
    queries = file.readlines()

import json
answer_file = "/projects/JHA/shared/dataset/mcq_filtered_by_match/answers.json"
with open(answer_file, "r") as file:
    true_answers = json.load(file)
response_mapping = {0:'A', 1:'B', 2:'C', 3:'D'}

idx = 79
print(queries[idx])
print(response_mapping[true_answers[idx]])

A 35-year old lady presented with dysphagia>> nocturnal asthma and weight loss for 6 years. The most probable diagnosis is -: A. Achalasia cardia, B. Lye stricture of esophagus, C. Gastroesophageal reflux diseae, D. Cancer esophagus. Please select the correct answer from A, B, C, D. Put your answer in \boxed{}.

A


## baseline id 100-1000

In [1]:
#174-175, 409-415 should be one
import re

log_file = "/scratch/gpfs/jx0800/data/graphrag/results/baseline_responses_2.txt"
with open(log_file, 'r') as file:
    output_content = file.read()

# Split the responses based on the separator
responses = output_content.strip().split("-" * 50)
print(len(responses))
print(responses[-2])

905

Query 1006: A 23-year-old woman has had altered consciousness and slurred speech for the past 24 hours. A head CT scan shows a right temporal hemorrhagic infarction. Cerebral angiography shows a distal right middle cerebral arterial occlusion. Within the past 3 years, she has had an episode of pulmonary embolism. A pregnancy 18 months ago ended in miscarriage. Laboratory studies show a false-positive serologic test for syphilis, normal prothrombin time (PT), elevated partial thromboplastin time (PTT), and normal platelet count. Which of the following is the most likely cause of these findings?: A. Antiphospholipid antibody, B. Disseminated intravascular coagulation, C. Factor V mutation, D. Hypercholesterolemia. Please select the correct answer from A, B, C, D. Put your answer in \box{}.
Response: The answer is: \boxed{A}.
Analysis: The patient's history of recurrent thrombotic events, including a pulmonary embolism and a cerebral arterial occlusion, along with a history of miscar

In [2]:
responses = responses[:74] + responses[76:309] + responses[313:-1]
print(len(responses))

898


In [3]:

# Regular expression to find the part after 'SUCCESS: Local Search Response:'
success_pattern = re.compile(r'Response: (.*)', re.DOTALL)

# Regular expression to find the content inside \boxed{}
boxed_pattern = re.compile(r'\\boxed\{([^}]+)\}')
box_pattern = re.compile(r'\\box\{([^}]+)\}')

seed_answers = []
# Process each response
for i, response in enumerate(responses):
    print(f"Processing Response {i}:")
    
    # Find the part after 'SUCCESS: Local Search Response:'
    success_match = success_pattern.search(response)
    if success_match:
        success_text = success_match.group(1).strip()
        #print(f"SUCCESS Text: {success_text}")
        
        # Check if there's an answer in \boxed{}
        boxed_match = boxed_pattern.search(success_text)
        if boxed_match:
            answer = boxed_match.group(1)
            seed_answers.append(answer)
            #print(f"Answer found in \\boxed{{}}: {answer}")
        else:
            box_match = box_pattern.search(success_text)
            if box_match:
                answer = box_match.group(1)
                seed_answers.append(answer)
            else:
                print("No answer found in \\boxed{}.")
    else:
        print("No 'SUCCESS: Local Search Response:' found in this response.")

print(len(seed_answers))
print(seed_answers)

Processing Response 0:
Processing Response 1:
Processing Response 2:
Processing Response 3:
Processing Response 4:
Processing Response 5:
Processing Response 6:
Processing Response 7:
Processing Response 8:
Processing Response 9:
Processing Response 10:
Processing Response 11:
Processing Response 12:
Processing Response 13:
Processing Response 14:
Processing Response 15:
Processing Response 16:
Processing Response 17:
Processing Response 18:
Processing Response 19:
Processing Response 20:
Processing Response 21:
Processing Response 22:
Processing Response 23:
Processing Response 24:
Processing Response 25:
Processing Response 26:
Processing Response 27:
Processing Response 28:
Processing Response 29:
Processing Response 30:
Processing Response 31:
Processing Response 32:
Processing Response 33:
Processing Response 34:
Processing Response 35:
Processing Response 36:
Processing Response 37:
Processing Response 38:
Processing Response 39:
Processing Response 40:
Processing Response 41:
Pr

In [4]:
# Mapping of valid responses
response_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

# Function to process each response
def process_response(response):
    # Check if the response starts with A, B, C, or D
    if response[0] in response_mapping:
        # Extract the first character (A, B, C, or D)
        key = response[0]
        return response_mapping[key]
    else:
        # Handle invalid responses
        return 4

# Process the entire response list
processed_seed_responses = [process_response(response) for response in seed_answers]

print(len(processed_seed_responses))

898


In [6]:
query_file = "/projects/JHA/shared/dataset/mcq_filtered_by_match/queries.txt"
with open(query_file, "r") as file:
    queries = file.readlines()
print(len(queries))
print(queries[174])
print(queries[408])

1000
A 48-year-old Caucasian man, diagnosed with diabetes 5 years ago, comes to the physician because of palpitations, Physical examination shows a diffuse darkening of his skin and testicular atrophy. Laboratory studies show elevated liver function and elevated blood sugars. A liver biopsy shows significantly elevated iron levels A diagnosis of hereditary hemochromatosis is made. Which one of the following laboratory findings is seen in hereditary hemochromatosis?: A. Decreased serum ceruloplasmin, B. Decreased serum ferritin, C. Decreased serum iron, D. Decreased total iron-binding capacity. Please select the correct answer from A, B, C, D. Put your answer in \boxed{}.

A man presents with weakness, pain in upperabdomen, hyperpigmentation, arthritis,hyperglycemia and enlarged palpable liver. Mostprobable diagnosis is -: A. Haemochromatosis, B. Addison's disease, C. Insulin dependent diabetes mellitus, D. Cushing's syndrome. Please select the correct answer from A, B, C, D. Put your a

In [9]:
import json
answer_file = "/projects/JHA/shared/dataset/mcq_filtered_by_match/answers.json"
with open(answer_file, "r") as file:
    answers = json.load(file)

true_answers = answers[100:174]+answers[175:408]+answers[409:]

def accuracy_score(y_true, y_pred):
    correct = 0
    wrong_indices = []
    
    for idx, (true, pred) in enumerate(zip(y_true, y_pred)):
        if true == pred:
            correct += 1
        else:
            wrong_indices.append(idx)
    
    accuracy = correct / len(y_true) if y_true else 0
    return accuracy, wrong_indices

accuracy, wrong_indices = accuracy_score(true_answers, processed_seed_responses)

# Print the result
print(f"Accuracy: {accuracy:.2f}")
print("wrong_indices:", wrong_indices)

Accuracy: 0.90
wrong_indices: [2, 8, 15, 39, 46, 61, 66, 73, 84, 89, 93, 107, 109, 126, 138, 147, 151, 161, 170, 191, 209, 210, 219, 255, 257, 284, 285, 290, 293, 316, 336, 338, 340, 343, 346, 353, 359, 363, 370, 377, 384, 386, 389, 392, 410, 413, 415, 422, 427, 440, 473, 480, 485, 493, 512, 518, 521, 532, 535, 539, 552, 556, 559, 589, 594, 605, 606, 634, 644, 667, 674, 687, 715, 744, 784, 794, 796, 823, 826, 839, 845, 847, 848, 856, 857, 869, 872, 880, 889, 896]


808
